In [ ]:
from commonfunctions import *
from skimage import io
from skimage.color import rgba2rgb, rgb2gray
from skimage import filters
from skimage.feature import hog
import numpy as np
import cv2
import sys
import os
# from PHOG_V2 import *
from Faces import *
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn import svm
import pickle

from sklearn.datasets import fetch_olivetti_faces

In [ ]:
bunch = fetch_olivetti_faces(data_home=None, shuffle = False, return_X_y = False)
show_images(bunch.images[0:10])

In [ ]:
def preprocessImg(img):
    img_copy = np.copy(img)
    if len(img_copy.shape) != 2:
        img_copy = rgb2gray(img_copy)
    img_copy = cv2.resize(img_copy, dsize=(24, 24), interpolation=cv2.INTER_CUBIC) 
    return img_copy

In [ ]:
face_images = bunch.images
for face in face_images:
    face = preprocessImg(face)
print(bunch.images.shape)

In [ ]:
def getIntegralImage(img):
    new_img = np.zeros(img.shape)
#     new_img[i,j] = img[0,0]
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            new_img[i,j] = np.sum(img[0:i+1,0:j+1])
    return new_img

In [ ]:
test_for_integral = np.ones((3,3))
test_for_integral[0,2] = 4
test_for_integral[1,1] = 9
test_for_integral[1,0] = 100
print(test_for_integral)
test_for_integral = getIntegralImage(test_for_integral)
print(test_for_integral)

In [ ]:
def calculateSumOfIntegralImage(int_img,indX,indY,width = 0,height = 0):
    if indX - width >= 0 and indY - height >= 0:
        return int_img[indX,indY] + int_img[indX - width,indY - height] - int_img[indX - width,indY] - int_img[indX,indY - height]
    elif indX - width < 0 and indY - height >= 0:
        return int_img[indX,indY] - int_img[indX,indY - height]
    elif indX - width >= 0 and indY - height < 0:
        return int_img[indX,indY] - int_img[indX - width,indY]
    else:
        return int_img[indX,indY]

In [ ]:
print(calculateSumOfIntegralImage(test_for_integral,0,0,1,1))
print(calculateSumOfIntegralImage(test_for_integral,2,2,3,3))
print(calculateSumOfIntegralImage(test_for_integral,2,2,2,1))
print(calculateSumOfIntegralImage(test_for_integral,1,2,0,0))
print(calculateSumOfIntegralImage(test_for_integral,2,0,1,1))
print(calculateSumOfIntegralImage(test_for_integral,2,2,0,0))

In [ ]:
def getFeatureArray(size = 24):
    feature_array = []
    loop_on_array = np.zeros((size,size))
    feature_names = ["Edge_Horizontal","Edge_Vertical","Line_Horizontal","Line_Verticle","Rectangle"]
    for name in feature_names:
                
        if name == "Edge_Horizontal":
            width = 1
            height = 0
            while not(width >= size and height >= size):
                for i in range(size - width):
                    for j in range(size - height):
#                         print((i,j),(i + width,j + height))
                        feature_array.append([(i,j),(i + width,j + height),name])
#                 print(len(feature_array), width, height)
#                 input()
                if width < size:
                    width += 2
                elif height != size:
                    width = 1
                    height += 1

        elif name == "Edge_Vertical":
            width = 0
            height = 1
            while not(width >= size and height >= size):
                for i in range(size - width):
                    for j in range(size - height):
#                         print((i,j),(i + width,j + height))
                        feature_array.append([(i,j),(i + width,j + height),name])
                if height < size:
                    height += 2
                elif width != size:
                    height = 1
                    width += 1

        elif name == "Line_Horizontal":
            width = 2
            height = 0
            while not(width >= size and height >= size):
                for i in range(size - width):
                    for j in range(size - height):
#                         print((i,j),(i + width,j + height))
                        feature_array.append([(i,j),(i + width,j + height),name])
                if width < size:
                    width += 3
                elif height != size:
                    width = 2
                    height += 1     
                    
        elif name == "Line_Verticle":
            width = 0
            height = 2
            while not(width >= size and height >= size):
                for i in range(size - width):
                    for j in range(size - height):
#                         print((i,j),(i + width,j + height))
                        feature_array.append([(i,j),(i + width,j + height),name])
                if height < size:
                    height += 3
                elif width != size:
                    height = 2
                    width += 1  

        elif name == "Rectangle":
            width = 1
            height = 1
            while not(width >= size and height >= size):
                for i in range(size - width):
                    for j in range(size - height):
                        feature_array.append([(i,j),(i + width,j + height),name])
                if width < size:
                    width += 2
                elif height != size:
                    width = 1
                    height += 2

    return feature_array

feature_array = getFeatureArray(24)
print(len(feature_array))
# print((feature_array))
                    
                    

In [ ]:
def calculateSumOfIntegralImage_V2(int_img,st_x,st_y,end_x,end_y):
    if (st_x-1 < 0) or (st_y-1) < 0:
        return int_img[end_x,end_y]
    if (end_x == 24):
        end_x -= 1
    if (end_y == 24):
        end_y -= 1
    if (st_x - 1 == 24):
        st_x -= 2
    if (st_y - 1 == 24):
        st_y -= 2
    
    return int_img[end_x,end_y] + int_img[st_x-1,st_y-1] - int_img[end_x,st_y-1] - int_img[st_x-1,end_y]

In [ ]:
def harrEdgeHorizontal(int_img,feature_params, scale_factor):
    # print(feature_params)
    st_x, st_y = feature_params[0] 
    end_x, end_y = feature_params[1] 
    st_x = int(st_x * scale_factor)
    st_y = int(st_y * scale_factor)
    end_x = int(end_x * scale_factor)
    end_y = int(end_y * scale_factor)
    middle_x = (end_x + st_x)//2
    middle_y = (end_y + st_y)//2
    up_value = calculateSumOfIntegralImage_V2(int_img,st_x,st_y,middle_x,end_y)
    down_value = calculateSumOfIntegralImage_V2(int_img,middle_x + 1,st_y,end_x,end_y)
    return abs(up_value - down_value)

In [ ]:
def harrEdgeVerticle(int_img,feature_params, scale_factor):
    # print(feature_params)
    st_x, st_y = feature_params[0] 
    end_x, end_y = feature_params[1] 
    st_x = int(st_x * scale_factor)
    st_y = int(st_y * scale_factor)
    end_x = int(end_x * scale_factor)
    end_y = int(end_y * scale_factor)
    middle_x = (end_x + st_x)//2
    middle_y = (end_y + st_y)//2
    left_value = calculateSumOfIntegralImage_V2(int_img,st_x,st_y,end_x,middle_y)
    right_value = calculateSumOfIntegralImage_V2(int_img,st_x,middle_y + 1,end_x,end_y)
    return abs(left_value - right_value)

In [ ]:
def harrLineHorizontal(int_img,feature_params, scale_factor):
    # print(feature_params)
    st_x, st_y = feature_params[0]
    end_x, end_y = feature_params[1] 
    st_x = int(st_x * scale_factor)
    st_y = int(st_y * scale_factor)
    end_x = int(end_x * scale_factor)
    end_y = int(end_y * scale_factor)
    third_x = (end_x + st_x + st_x)//3
    third_2x = (end_x + end_x + st_x)//3
    third_y = (end_y + st_y + st_y)//3
    third_2y = (end_y + end_y + st_y)//3
#     print("st_x,st_y,third_x,third_y,third_2x,third_2y,end_x,end_y")
#     print(st_x,st_y,third_x,third_y,third_2x,third_2y,end_x,end_y)
    up_value = calculateSumOfIntegralImage_V2(int_img,st_x,st_y,third_x,end_y)
    middle_value = calculateSumOfIntegralImage_V2(int_img,third_x + 1,st_y,third_2x,end_y)
    down_value = calculateSumOfIntegralImage_V2(int_img,third_2x + 1,st_y,end_x,end_y)
    return abs(up_value + down_value - middle_value)

In [ ]:
def harrLineVerticle(int_img,feature_params, scale_factor):
    # print(feature_params)
    st_x, st_y = feature_params[0] 
    end_x, end_y = feature_params[1] 
    st_x = int(st_x * scale_factor)
    st_y = int(st_y * scale_factor)
    end_x = int(end_x * scale_factor)
    end_y = int(end_y * scale_factor)
    third_x = (end_x + st_x + st_x)//3
    third_2x = (end_x + end_x + st_x)//3
    third_y = (end_y + st_y + st_y)//3
    third_2y = (end_y + end_y + st_y)//3
    left_value = calculateSumOfIntegralImage_V2(int_img,st_x,st_y,end_x,third_y)
    middle_value = calculateSumOfIntegralImage_V2(int_img,st_x,third_y + 1,end_x,third_2y)
    right_value = calculateSumOfIntegralImage_V2(int_img,st_x,third_2y + 1,end_x,end_y)
    return abs(left_value + right_value - middle_value)

In [ ]:
def harrRectangle(int_img,feature_params,scale_factor):
    # print(feature_params)
    try:
        st_x, st_y = feature_params[0]
        end_x, end_y = feature_params[1] 
        st_x = int(st_x * scale_factor)
        st_y = int(st_y * scale_factor)
        end_x = int(end_x * scale_factor)
        end_y = int(end_y * scale_factor)
        middle_x = (end_x + st_x)//2
        middle_y = (end_y + st_y)//2
        up_left_value = calculateSumOfIntegralImage_V2(int_img,st_x,st_y,middle_x,middle_y)
        up_right_value = calculateSumOfIntegralImage_V2(int_img,st_x,middle_y + 1,middle_x,end_y)
        down_left_value = calculateSumOfIntegralImage_V2(int_img,middle_x + 1,st_y,end_x,middle_y)
        down_right_value = calculateSumOfIntegralImage_V2(int_img,middle_x + 1,middle_y + 1,end_x,end_y)
#         print(up_left_value,up_right_value,down_left_value,down_right_value,up_left_value + down_right_value - up_right_value - down_left_value)
        return abs(up_left_value + down_right_value - up_right_value - down_left_value)
    except:
        print(feature_params)


In [ ]:
def calculateHaar(int_img,feature_params, scale_factor):
    try:
        if feature_params[2] == "Edge_Horizontal":
            return harrEdgeHorizontal(int_img,feature_params[0:2],scale_factor)
        elif feature_params[2] == "Edge_Vertical":
            return harrEdgeVerticle(int_img,feature_params[0:2],scale_factor)
        elif feature_params[2] == "Line_Horizontal":
            return harrLineHorizontal(int_img,feature_params[0:2],scale_factor)
        elif feature_params[2] == "Line_Verticle":
            return harrLineVerticle(int_img,feature_params[0:2],scale_factor)
        elif feature_params[2] == "Rectangle":
            return harrRectangle(int_img,feature_params[0:2],scale_factor)
    except:
        print(feature_params)
        print(feature_params[0] * scale_factor)
        

In [ ]:
# for test
test = getIntegralImage(preprocessImg(bunch.images[0]))
print(test)
print(test.shape)

In [ ]:
print(feature_array[0])
print(calculateHaar(test,feature_array[0],1))

In [ ]:
data = np.copy(bunch.images)
labels = [1 for i in range(len(bunch.images))]

In [ ]:
def ReadFile(path,label,imgs,labels):
    files = os.listdir(path)
    num_images = len(files)
    count = 0
    for f in files:
        img = io.imread(os.path.join(path, f), as_gray=False)
        img= preprocessImg(img)
        imgs.append(img)
        labels.append(label)

In [ ]:
path = "./FaceDetection/False/NotHuman"
imgs = []
labels = []
ReadFile(path,0,imgs,labels)
print(len(imgs))
print(len(labels))
show_images(imgs[0:10])
# print(False_labels)

In [ ]:
# imgs = np.array(imgs)
# print(imgs.shape)

for i in range (data.shape[0]):
    imgs.append(preprocessImg(data[i]))
    labels.append(1)
# imgs = np.array(imgs)
show_images(imgs[411:421])
print(labels[411:len(labels)])

In [ ]:
# print(len(feature_array))
# print(len(imgs))
# feat = np.ones((len(feature_array),len(imgs)))
# print(feat.shape)

# int_imgs = []
# for img in imgs:
#     int_imgs.append(getIntegralImage(img))

# count = 0
# for i in range(len(feature_array)):
#     print(count)
#     count += 1
#     for j in range(len(int_imgs)):
#         feat[i][j] = calculateHaar(int_imgs[j],feature_array[i])
# print(feat.shape)
# print(feat[1000:1050])

In [ ]:
class Stumb:
    def __init__(self):
        self.more = True
        self.feature_idx = None
        self.threshold = None
        self.alpha = None
    
    def predict(self,img_index, feature_array,feat):
        # res = calculateHaar(int_img,feature_array[self.feature_idx])
        res = feat[self.feature_idx,img_index]
        if (res >= self.threshold and self.more == True) or (res < self.threshold and self.more == False) :
            return 1
        else:
            return 0

    def predictOnInt_img(self,int_img, feature_array,feat, scale_factor):
        res = calculateHaar(int_img,feature_array[self.feature_idx],scale_factor)
        # res = feat[self.feature_idx,img_index]
        if (res >= self.threshold and self.more == True) or (res < self.threshold and self.more == False) :
            return 1
        else:
            return 0
        
    def predictOnArray(self,int_imgs, feature_array, feat, scale_factor):
        predictions = np.ones((len(int_imgs)))
        for i in range(len(int_imgs)):
            predictions[i] = self.predictOnInt_img(int_imgs[i],feature_array,feat, scale_factor)
        return predictions
    
    def getIndex(self):
        return self.feature_idx

In [ ]:
class AdaBoost:
    def __init__(self,n_clf = 3, n_features = 160000):
        self.n_features = n_features
        self.n_clf = n_clf
        self.clfs = []
        self.indicies = []
    
    def train(self,imgs,labels,feature_array,feat,int_imgs,st_ind,end_ind):
        weight= np.full(len(imgs), (1 / len(imgs)))
        count = 0
        lbls = np.array(labels)
        while count < self.n_clf:
            count += 1
            classifier = Stumb()
            min_error = float("inf")
            int_img = []
            for img in imgs:
                int_img.append(getIntegralImage(img))
            for f in range(st_ind,end_ind,1):
                # if f == 34240:
                #     print("Hi")
                #     input()
                if f in self.indicies:
                    # print("I am here")
                    continue
                f_values = np.copy(feat[f])
                # print(count,f)
#                 for img in int_img:
# #                     print(f_values,f)
#                     f_values.append([calculateHaar(img,feature_array[f])])
#                 f_values=np.array(f_values)
                thresholds = np.unique(f_values)
                if thresholds.shape[0] == 1:
                    continue
                for trsh in thresholds:
                    more = True
                    predictions = np.ones((f_values.shape[0]))
                    predictions[f_values < trsh] = 0
                    error = np.sum(weight[lbls != predictions])
                    
                    if error > 0.5:
                        error = 1 - error
                        more = False
                    
                    if error < min_error:
                        classifier.more = more
                        classifier.feature_idx = f
                        classifier.threshold = trsh
                        min_error = error 
            beta = min_error + 0.0000000001 / (1 - min_error + 0.0000000001)
            preds = classifier.predictOnArray(int_imgs, feature_array, feat,1)
            predres = np.ones((lbls.shape[0]))
            predres[lbls == preds] = 0
            predres[lbls != preds] = 1
#             print(weight)
#             print(beta)
#             print(predres)
            weight *= (beta ** predres)
#             print(weight)
#             print(np.sum(weight))
            weight = weight / np.sum(weight)
#             predictions = clf.predict(X)
#             print(weight)
#             print("----------------------------------------")
            classifier.alpha = np.log(1/beta)
            classifier.alpha = 0.5 * classifier.alpha
            self.clfs.append(classifier)
            self.indicies.append(classifier.feature_idx)
            # EPS = 1e-10
            # classifier.alpha = 0.5 * np.log((1.0 - min_error + EPS) / (min_error + EPS))

            # # calculate predictions and update weights
            # predictions = classifier.predictOnArray(int_imgs,feature_array,feat,1)
            # predictions[predictions == 0] = - 1

            # weight *= np.exp(-classifier.alpha * lbls * predictions)
            # # Normalize to one
            # weight /= np.sum(weight)

            # # Save classifier
            # self.clfs.append(classifier)
            
    def getClassifierIndices(self , c):
        with open('filename' + str(c) +'.txt', 'w') as f:
            for clf in self.clfs:
                print(clf.getIndex(), file=f)
                print(clf.alpha, file=f)
                print(clf.threshold, file=f)
                if clf.more == True:
                    print(1, file=f)
                else:
                    print(0, file=f)

    def predict(self, img, feat, scale_factor):
        imt_img = getIntegralImage(img)
        clf_preds = [clf.alpha * clf.predictOnInt_img(imt_img,feature_array, feat, scale_factor) for clf in self.clfs]
        y_pred = np.sum(clf_preds, axis=0)
        y_pred = np.sign(y_pred)

        return y_pred

In [ ]:
# ada = AdaBoost(1,len(feature_array))

In [ ]:
# croped_img = []
# croped_img += (imgs[0:50] + imgs[500:550])
# # croped_img.append(imgs[0:100])
# # croped_img.append(imgs[500:600])
# lbls = []
# lbls += (labels[0:50] + labels[500:550])
# # lbls.append(labels[0:100])
# # lbls.append(labels[500:600])
# # ada.train(imgs,labels,feature_array)
# ada.train(imgs,labels,feature_array,feat,int_imgs)
# # print(len(croped_img),len(lbls))

In [ ]:
# ada.getClassifierIndices()

In [ ]:
# preds = []
# true = []
# for i in range(200,411):
#     # print(i,ada.predict(imgs[i]))
#     preds.append(ada.predict(imgs[i],feat))
#     true.append(labels[i])
# for i in range(700,811):
#     # print(i,ada.predict(imgs[i]))
#     preds.append(ada.predict(imgs[i],feat))
#     true.append(labels[i])
# preds = np.array(preds)
# true = np.array(true)
# acc = np.zeros((preds.shape[0]))
# acc[preds == true] = 1
# acc[preds != true] = 0
# print(np.sum(acc)/acc.shape[0])

In [ ]:
# print(len(feature_array))
# print(len(imgs))

croped_img = []
croped_img += (imgs[0:100] + imgs[500:600])

feat = np.ones((len(feature_array),len(croped_img)))
# print(feat.shape)

lbls = []
lbls += (labels[0:100] + labels[500:600])

int_imgs = []
for img in croped_img:
    int_imgs.append(getIntegralImage(img))

count = 0
for i in range(len(feature_array)):
    # print(count)
    # count += 1
    for j in range(len(int_imgs)):
        feat[i][j] = calculateHaar(int_imgs[j],feature_array[i],1)
print(feat.shape)
# print(feat[1000:1050])

In [ ]:
# ada = AdaBoost(10,len(feature_array))

In [ ]:
# ada.train(croped_img,lbls,feature_array,feat,int_imgs)

In [ ]:
# ada.getClassifierIndices()

In [ ]:
count = 10
for i in range(0,len(feature_array) - int(len(feature_array) /100),int(len(feature_array) /100)):
    adbst = AdaBoost(10,len(feature_array))
    adbst.train(croped_img,lbls,feature_array,feat,int_imgs,i,i + int(len(feature_array) /100))
    adbst.getClassifierIndices(count)
    count += 1
    # if(count == 12):
    #     break

In [ ]:
# preds = []
# true = []
# for i in range(200,411):
#     # print(i,ada.predict(imgs[i]))
#     preds.append(ada.predict(imgs[i],feat,1))
#     true.append(labels[i])
# for i in range(700,811):
#     # print(i,ada.predict(imgs[i]))
#     preds.append(ada.predict(imgs[i],feat,1))
#     true.append(labels[i])
# preds = np.array(preds)
# true = np.array(true)
# acc = np.zeros((preds.shape[0]))
# acc[preds == true] = 1
# acc[preds != true] = 0
# print(np.sum(acc)/acc.shape[0])

In [ ]:
# ada.getClassifierIndices()

In [ ]:
# test_img = io.imread("./images/1.jpg")
# test_img = rgb2gray(test_img)
# show_images([test_img])
# print(test_img.shape)

In [ ]:
# def slideWindow(test_img,adaBoost : AdaBoost):
#     winX = 384
#     winY = 384
#     scale = 16
#     results = []
#     while (winX < test_img.shape[0]) and (winY < test_img.shape[1]): 
#         # print(winX,winY)
#         for i in range(0,int(test_img.shape[0] - winX),int(winX/4)):
#             for j in range(0,int(test_img.shape[1] - winY),int(winY/4)):
#                 # print(i , int(i+winX) ,j ,int(j + winY))
#                 croppedImg = np.copy(test_img[i:int(i+winX),j:int(j+winY)])
#                 val = adaBoost.predict(croppedImg,feat,scale)
#                 if val == 1:
#                     results.append(croppedImg)
#         scale = scale * 2
#         winX *= 2
#         winY *= 2
#         # print("I am here")
#     return results

In [ ]:
# results = slideWindow(test_img,ada)

In [ ]:
# print(len(results))
# show_images(results)
# # show_images(results[12300:12310])

In [ ]:
# def cascadeFind(test_img,adaBoost : AdaBoost):
#     winX = 384
#     winY = 384
#     scale = 16
#     results = []
#     # scores = []
#     while (winX < test_img.shape[0]) and (winY < test_img.shape[1]): 
#         print(winX,winY)
#         for i in range(0,int(test_img.shape[0] - winX),int(winX/4)):
#             for j in range(0,int(test_img.shape[1] - winY),int(winY/4)):
#                 # print(i , int(i+winX) ,j ,int(j + winY))
#                 croppedImg = np.copy(test_img[i:int(i+winX),j:int(j+winY)])
#                 int_img = getIntegralImage(croppedImg)
#                 # score = 0
#                 val = 1
#                 for k in range(len(adaBoost.indicies)):
#                     # print(k,adaBoost.clfs[k].predictOnInt_img(int_img, feature_array,feat, scale))
#                     res = adaBoost.clfs[k].predictOnInt_img(int_img, feature_array,feat, scale)
#                     # print(res)
#                     val *= res
#                     if val == 0:
#                         break
#                     # score += adaBoost.clfs[k].alpha
#                 if val == 1:
#                     results.append(croppedImg)
#                     # scores.append(score)
#         scale = scale * 2
#         winX *= 2
#         winY *= 2
#         # print("I am here")
#     return results 

In [ ]:
# test_img = io.imread("./images/1.jpg")
# test_img = rgb2gray(test_img)
# results = cascadeFind(test_img,ada)

In [ ]:
# print(len(results))
# # print(len(scores))
# # print(np.max(scores))

In [ ]:
# f_values = []
# f_values.append([calculateHaar(getIntegralImage(imgs[0]),feature_array[0])])
# f_values.append([calculateHaar(getIntegralImage(imgs[0]),feature_array[1])])
# f_values.append([calculateHaar(getIntegralImage(imgs[0]),feature_array[2])])
# f_values=np.array(f_values)
# thresholds = np.unique(f_values)
# print(thresholds.shape)
# for trsh in thresholds:
#     print(trsh)

In [ ]:
# f_values = []

In [ ]:
# weight = np.full(4,0.25)
# print(weight)
# beta = 0.25 / (1 - 0.25)
# preds = np.ones((4))
# preds[0] = 0
# preds[3] = 0
# lbls = np.ones((4))
# lbls[3] = 0
# predres = np.ones((lbls.shape[0]))
# predres[lbls == preds] = 1
# predres[lbls != preds] = 0
# print(predres)
# print(beta)
# # weight *= np.exp(beta,predres)
# print(weight)
# print((beta ** predres))
# weight *= (beta ** predres)
# print(weight)
# weight = weight / np.sum(weight)
# print(weight)
# print(np.log(1/beta))